In [1]:
from Arbeitsbibliothek.myshadow import ShadowShot, ShadowCollection
import networkx as nx
import torch
import torch.nn as nn
import pytorch_lightning as pl
#from torchviz import make_dot
#from torch.utils.tensorboard import SummaryWriter
import numpy as np
from Arbeitsbibliothek.myshadow import ShadowShot, ShadowCollection
import random
from Arbeitsbibliothek.mytoolbox import make_dia
import matplotlib.pyplot as plt
import os
import pprint
from Arbeitsbibliothek.qiskit_qaoa import run_qaoa, run_single_qaoa_cir
import networkx as nx
import torch.nn.functional as F
import sys

def cluster_string(ersatz_string=""):
    try:
        name = sys.argv[2]
    except IndexError:
        name = ersatz_string
    return name

def verbinde_zu_array(a, b):
    # Prüfe, ob 'a' oder 'b' ein NumPy-Array ist und wandle entsprechend um
    if isinstance(a, list):
        a = np.array(a)
    if isinstance(b, list):
        b = np.array(b)
    
    return np.concatenate((a, b))

#os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
lauf_variable = 0
loss_index = []
loss_curve = []
def log_loss(loss):
    global lauf_variable
    loss_curve.append(loss.item())
    loss_index.append(lauf_variable)
    lauf_variable += 1

class BitGeneratorModel(pl.LightningModule):
    def __init__(self, input_dim, hidden_size = 2):
        super().__init__()
        output_size = 1
        self.lstm = nn.LSTM(input_dim + hidden_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        self.lr = 0.01

    def forward(self, x, prev_bits):
        
        # Kombinieren des Inputs und der bisher generierten Bits
        '''
        tensor1_expanded = x.unsqueeze(1).expand(-1, prev_bits.size(1), -1)
        tensor2_expanded = prev_bits.unsqueeze(2)
        combined_input = torch.cat((tensor1_expanded, tensor2_expanded), dim=2)
        print("jdjd", combined_input.size())
        '''
        if prev_bits.size(1) > 0:  # Überprüfen, ob prev_bits eine positive Sequenzlänge hat
            tensor1_expanded = x.unsqueeze(1).expand(-1, prev_bits.size(1), -1)
            tensor2_expanded = prev_bits.unsqueeze(2)
            combined_input = torch.cat((tensor1_expanded, tensor2_expanded), dim=2)
        else:
            # Erstellen eines Dummy-Tensors mit Sequenzlänge 1, um das LSTM zu befriedigen
            #print(x)
            combined_input = x.unsqueeze(1)
            #print(combined_input)
            #testsdf
        # Berechnen der Wahrscheinlichkeit für den nächsten Bit
        output, (hn, cn) = self.lstm(combined_input)
        output = output[:,-1,:]

        prob = self.linear(output)  # Nehmen Sie nur den letzten Ausgang für jede Sequenz
        prob = self.sigmoid(prob)
        #prob = prob.squeeze(-1)
        return prob
        

    def training_step(self, batch, batch_idx):
        x, y = batch
        batch_size, sequence_length = y.shape
        #loss_fn = nn.BCELoss(reduction='mean')
        loss_fn = torch.nn.SmoothL1Loss(reduction='mean')
        #loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
        #loss_fn = torch.nn.PoissonNLLLoss(reduction='mean', full=True)
        loss = 0
        prev_bits = torch.zeros((batch_size, 0), device=self.device)
        for t in range(sequence_length):
            prob = self(x, prev_bits)
            y_t = y[:, t:t+1]  # Ziel-Bit für den aktuellen Zeitschritt
            #### Test
            y_string = y[:, 0:t+1]
            y_nn = y_string
            if t == 0:
                y_nn = prob
            else:
                y_nn = torch.cat((prev_bits, prob), dim=1)
            ####
                
            #loss += loss_fn(prob, y_t)
            loss += loss_fn(y_nn, y_string)
            # Aktualisieren der bisher generierten Bits mit dem Ziel-Bit
            prev_bits = torch.cat((prev_bits, y_t), dim=1)
        log_loss(loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)


'''
max_iter = 400
N = 100
param = 2
bits = 4
x = torch.rand(N, param)
y = torch.randint(0, 2, (N, bits), dtype=torch.float)

dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x, y), batch_size=N)
    
#return dataloader, input_size, hidden_size

model = BitGeneratorModel(param)
trainer = pl.Trainer(max_epochs=max_iter, accelerator='auto', devices='auto')
trainer.fit(model, dataloader)
'''

cluster_string()

QAOAenergie = 0
G = nx.Graph()
G.add_nodes_from([0, 1, 2, 3])
#G.add_nodes_from([0, 1, 2, 3, 4, 5, 6, 7])
#G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 0)])
#G.add_nodes_from([0, 1, 2, 3, 4, 5])
#G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 4), (4, 5),(5,0)])
#G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 4), (4, 5),(5,6), (6, 7), (7,0)])
G.add_edges_from([(0, 2), (1, 2), (1, 3), (2, 3)]) # Random Graph 1 - LSG: -3 und [0010], [0110], [0011]
#G.add_edges_from([(0, 2), (1, 2), (1, 3)]) # Random Graph 2 - LSG: -3 und [0011]
#G.add_edges_from([(0, 1), (0, 2), (1, 2), (2, 3)]) # Random Graph 3 - LSG: -3 und [0010], [0110], [0101]

def generate_bitstrings(model, x, N, L):
    prev_bits = torch.zeros((N, 0))
    for t in range(L):
        prob = model(x, prev_bits)
        # Aktualisieren der bisher generierten Bits mit dem Ziel-Bit
        prev_bits = torch.cat((prev_bits, prob), dim=1)
        prev_bits = torch.bernoulli(prev_bits)
    bits = prev_bits #torch.bernoulli(prev_bits)
    #bits = (prev_bits>0.5).float()
    return bits.detach().cpu().numpy()

def create_data(betas, gammas, n=100, max_iter=1000, showDia=False):
    if not isinstance(betas , list): raise ValueError('Parameter müssen Listen sein, keine Arrays')
    parameters = betas + gammas
    
    global G
    bitlength = G.number_of_nodes()
    #input_size = 2
    gw = nx.to_numpy_array(G)
    res, shadow, expec= run_single_qaoa_cir(G, parameters)
    global QAOAenergie
    QAOAenergie = shadow.get_energy_list()[0]
    shadow.eliminated_equivalent()
    
    print("QAOA Energie: ", QAOAenergie)
    Y = shadow.shadowList[0].shadow
    #Y = [[1,0,1,0] for _ in range(1000)]

    meanY = np.mean(Y, axis=0)
    print("Durchschnitt pro Bit (QAOA)", meanY)
    
    X = [parameters for i in range(len(Y))]
    N = len(Y)
    x = torch.tensor(X, dtype=torch.float32)
    #x = torch.zeros_like(x)
    #x = torch.ones_like(x)
    #x = torch.rand(N, len(parameters))
    y = torch.tensor(Y, dtype=torch.float32)
    

    dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x, y), batch_size=N)
    num_param = len(parameters)
    model = BitGeneratorModel(num_param, hidden_size=4) # MyPixelRNN(num_param, hidden_size)
    trainer = pl.Trainer(max_epochs=max_iter, accelerator='auto', devices='auto')
    '''
    tuner = pl.tuner.tuning.Tuner(trainer)
    lr_finder = tuner.lr_find(model,
                                    train_dataloaders=dataloader,
                                    min_lr=0.000000001,
                                    max_lr=1.0,
                                    early_stop_threshold=None,
                                    num_training=max_iter) # max epoche (maximal 1000)
    fig = lr_finder.plot(suggest=True)
    plt.title("LR Finder")
    fig.show()
    model.hparams.lr = lr_finder.suggestion()
    '''
    trainer.fit(model, dataloader)

    if(showDia):
        plt.figure()
        plt.title("Loss")
        plt.plot(loss_index, loss_curve)
        plt.show()
    array = generate_bitstrings(model, x, N, bitlength)
    meanNN = np.mean(array, axis=0)
    print("Durchschnitt pro Bit (NN)", meanNN)
    gw = nx.to_numpy_array(G)
    
    NNshadow = ShadowCollection([ShadowShot(array, parameters)], gw)
    NNenergie = NNshadow.get_energy_list()[0]

    print("Energie von NN", NNenergie)

    return NNenergie, QAOAenergie

def create_data_QAOA(layers = 1, max_iter = 100, shots=1024, randomGraph=False, learning_sample_cap=None, split_mix=None, callByExp=None, saveBitsString=None):
    global G
    bitlength = G.number_of_nodes()
    if randomGraph:
        G = nx.gnp_random_graph(bitlength, 0.5)
        for (u, v) in G.edges():
            G.edges[u,v]['weight'] = 1
    gw = nx.to_numpy_array(G)
    res, shadow, expec= run_qaoa(G, layers=layers, shots=shots, callByExp=callByExp)
    #global QAOAenergie
    QAOAenergies = shadow.get_energy_list()
    shadow.eliminated_equivalent()
    NNenergies = []
    param_list = []
    for sh in shadow.shadowList:
        parameters = sh.parameters
        Y = sh.shadow
        if learning_sample_cap!=None:
            Y = Y[:learning_sample_cap]
        X = [parameters for i in range(len(Y))]
        N = len(Y)
        x = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(Y, dtype=torch.float32)

        dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x, y), batch_size=N)
        num_param = len(parameters)
        model = BitGeneratorModel(num_param, hidden_size=4) # MyPixelRNN(num_param, hidden_size)
        trainer = pl.Trainer(max_epochs=max_iter, accelerator='auto', devices='auto')
        trainer.fit(model, dataloader)

        array = generate_bitstrings(model, x, N, bitlength)
        meanNN = np.mean(array, axis=0)
        print("Durchschnitt pro Bit (NN)", meanNN)
        gw = nx.to_numpy_array(G)

        if split_mix != None:
            array = array[:split_mix[1]]
            array = list(array)
            array += Y[:split_mix[0]]
        
        NNshadow = ShadowCollection([ShadowShot(array, parameters)], gw)
        NNenergie = NNshadow.get_energy_list()[0]

        NNenergies.append(NNenergie)
        param_list.append(parameters)
    
    return QAOAenergies, NNenergies, param_list

def create_data_mit_abweichung_pool(konst_betas, konst_gammas, beta_grenzen, gamma_grenzen, random_sets=5, max_iter=600, showDia=False, savePath=None, rndMitAnaly=False):
    X = []
    Y = []
    global G
    bitlength = G.number_of_nodes()
    for i in range(random_sets):
        beta = random.uniform(beta_grenzen[0], beta_grenzen[1])
        gamma = random.uniform(gamma_grenzen[0], gamma_grenzen[1])
        if rndMitAnaly:
            konst_betas.append(beta)
            konst_gammas.append(gamma)
            konst_betas.sort()
            konst_gammas.sort()
        parameters = [beta, gamma]
        res, shadow, expec= run_single_qaoa_cir(G, parameters)
        shadow.eliminated_equivalent()
        Y_ = shadow.shadowList[0].shadow        
        X_ = [parameters for i in range(len(Y_))]

        X += X_
        if not isinstance(Y_, list): Y_ = Y_.tolist()
        Y += Y_
    x = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(Y, dtype=torch.float32)
    print(x.size(), y.size())
    N = len(Y)
    dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x, y), batch_size=N)
    num_param = len(parameters)
    model = BitGeneratorModel(num_param, hidden_size=4)
    #model = BitGeneratorModel2(num_param, hidden_size=4)
    trainer = pl.Trainer(max_epochs=max_iter, accelerator='auto', devices='auto')
    trainer.fit(model, dataloader)

    if(showDia):
        plt.figure()
        plt.title("Loss")
        plt.plot(loss_index, loss_curve)
        plt.show()
    para_list = []
    QAOAenergies = []
    NNenergies = []
    fileString = ""
    for beta, gamma in zip(konst_betas, konst_gammas):
        konst_parameters = (beta, gamma)
        res, shadow, expec= run_single_qaoa_cir(G, konst_parameters)
        shadow.eliminated_equivalent()        
        X_ = [parameters for i in range(len(Y))]
        x_ = torch.tensor(X, dtype=torch.float32)
        QAOAenergie = shadow.get_energy_list()[0]
        array = generate_bitstrings(model, x_, N, bitlength)
        gw = nx.to_numpy_array(G)
    
        NNshadow = ShadowCollection([ShadowShot(array, parameters)], gw)
        NNenergie = NNshadow.get_energy_list()[0]

        para_list.append(konst_parameters)
        QAOAenergies.append(QAOAenergie)
        NNenergies.append(NNenergie)

        if not savePath==None:
            fileString += str(QAOAenergie) +"\t" + str(NNenergie) +"\t"
            for p in konst_parameters:
                fileString += str(p) +"\t"
            fileString += "\n"
    if  not savePath==None:
        f = open(savePath, "w")
        f.write(fileString)
        f.close()

        
    return para_list, QAOAenergies, NNenergies

def create_data_mit_abweichung(betas, gammas, abweichung, n=100, max_iter=1000, showDia=False):
    if not isinstance(betas , list): raise ValueError('Parameter müssen Listen sein, keine Arrays')
    parameters = betas + gammas
    para_minus = [i - abweichung for i in parameters]
    para_plus = [i + abweichung for i in parameters]
    
    global G
    bitlength = G.number_of_nodes()
    #input_size = 2
    gw = nx.to_numpy_array(G)
    res, shadow, expec= run_single_qaoa_cir(G, parameters)
    _, shadow_minus, _= run_single_qaoa_cir(G, para_minus)
    _, shadow_plus, _= run_single_qaoa_cir(G, para_plus)
    global QAOAenergie
    QAOAenergie = shadow.get_energy_list()[0]
    QAOAenergie_p = shadow_plus.get_energy_list()[0]
    QAOAenergie_m = shadow_minus.get_energy_list()[0]
    shadow.eliminated_equivalent()
    
    print("QAOA Energie: ", QAOAenergie, QAOAenergie_p, QAOAenergie_m)
    print("QAOA Energie+: ", QAOAenergie_p)
    print("QAOA Energie-: ", QAOAenergie_m)
    Y = shadow.shadowList[0].shadow
    #Y = [[1,0,1,0] for _ in range(1000)]

    meanY = np.mean(Y, axis=0)
    print("Durchschnitt pro Bit (QAOA)", meanY)
    
    X = [parameters for i in range(len(Y))]
    X_minus = [para_minus for i in range(len(Y))]
    X_plus = [para_plus for i in range(len(Y))]
    N = len(Y)
    x = torch.tensor(X, dtype=torch.float32)
    x_plus = torch.tensor(X_plus, dtype=torch.float32)
    x_minus = torch.tensor(X_minus, dtype=torch.float32)
    
    y = torch.tensor(Y, dtype=torch.float32)
    

    dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x, y), batch_size=N)
    num_param = len(parameters)
    model = BitGeneratorModel(num_param, hidden_size=4) # MyPixelRNN(num_param, hidden_size)
    trainer = pl.Trainer(max_epochs=max_iter, accelerator='auto', devices='auto')
    
    trainer.fit(model, dataloader)

    array = generate_bitstrings(model, x, N, bitlength)
    array_plus = generate_bitstrings(model, x_plus, N, bitlength)
    array_minus = generate_bitstrings(model, x_minus, N, bitlength)

    
    gw = nx.to_numpy_array(G)
    
    NNshadow = ShadowCollection([ShadowShot(array, parameters)], gw)
    NNshadow_plus = ShadowCollection([ShadowShot(array_plus, para_plus)], gw)
    NNshadow_minus = ShadowCollection([ShadowShot(array_minus, para_minus)], gw)
    NNenergie = NNshadow.get_energy_list()[0]
    NNenergie_p = NNshadow_plus.get_energy_list()[0]
    NNenergie_m = NNshadow_minus.get_energy_list()[0]

    print("Energie von NN", NNenergie)
    print("Energie von NN+",NNenergie_p)
    print("Energie von NN-", NNenergie_m)

    return NNenergie, QAOAenergie

problem_gamma = (3 * np.pi / 4) * 0.75 + np.pi * 0.25

# Beispiel mit Graph
#create_data([2], [2], max_iter=400, showDia=True)
#create_data_mit_abweichung([2], [2], abweichung=0.05, max_iter=400, showDia=True)
#create_data_QAOA()

# gibt File aus mit konstanten beta und gamma variert
def writefile(n=1, epoch=400):
    #logging.getLogger('lightning').setLevel(0)
    fileString = ""
    gammas = np.linspace(0, np.pi, n)
    i = 1
    for gamma in gammas:
        print("Beginn von Nummer: ", i)
        i += 1
        NNenergie, QAOAenergie = create_data([2], [gamma], max_iter=epoch, showDia=False)
        fileString += str(QAOAenergie) +"\t" + str(NNenergie) +"\t" + str(gamma) + "\n"
    file_name = "LSTM beta 2 hidden 4 bits 4 Random Graph3_2.txt"
    file_name = cluster_string(file_name)
    f = open(file_name, "w")
    f.write(fileString)
    f.close()
    print("fertig!!")

# gibt FIle aus für QAOA
def writefile_QAOA(n=1, epoch=400, layers=1, shots=1024, learning_sample_cap=None, split_mix=None, saveBitsString=False):
    fileString = ""
    QAOAenergies, NNenergies, param_list = create_data_QAOA(max_iter=epoch, layers=layers, shots=shots, learning_sample_cap=learning_sample_cap, split_mix= split_mix)
    i = 1
    for q,n,ps in zip(QAOAenergies, NNenergies, param_list):
        print("Beginn von Nummer: ", i)
        i += 1
        fileString += str(q) +"\t" + str(n) +"\t"
        for p in ps:
            fileString += str(p) +"\t"
        fileString += "\n"
    file_name = "LSTM beta QAOA hidden 4 bits 8 layer1_5.txt"
    file_name = cluster_string(file_name)
    f = open(file_name, "w")
    f.write(fileString)
    f.close()
    print("fertig!!")

def writefile_QAOA_training_plus_NN(n=1, epoch=600, layers=1, shots=1024, add_NN_samples=None, max_iter=None, fileprefix=""):
    if add_NN_samples==None: add_NN_samples=shots
    def unterfunktion(expec, G, theta, result, shadow, running_index):
        #print(running_index)
        
        bitlength = G.number_of_nodes()
        Y = shadow
        X = [theta for i in range(len(Y))]
        N = len(Y)
        x = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(Y, dtype=torch.float32)
        max_iter=epoch

        dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x, y), batch_size=N)
        num_param = len(theta)
        model = BitGeneratorModel(num_param, hidden_size=4) # MyPixelRNN(num_param, hidden_size)
        trainer = pl.Trainer(max_epochs=max_iter, accelerator='auto', devices='auto')
        trainer.fit(model, dataloader)

        gw = nx.to_numpy_array(G)
        
        array = generate_bitstrings(model, x, N, bitlength)
        shadow += list(array[:add_NN_samples])
        NNshadow = ShadowCollection([ShadowShot(shadow, theta)], gw)
        NNexpec = NNshadow.get_energy_list()[0]


        return NNexpec, shadow

    fileString = ""
    res1, shadow1, expec1 = run_qaoa(G, layers=layers, shots=shots, callByExp=unterfunktion, max_iter=max_iter) # QAOA Simualtion gemischt mit NN
    res2, shadow2, expec2 = run_qaoa(G, layers=layers, shots=shots, seed=None, max_iter=max_iter) # QAOA Simualtion ohne NN
    res3, shadow3, expec3 = run_qaoa(G, layers=layers, shots=shots + add_NN_samples, max_iter=max_iter) # QAOA Simualtion ohne NN, aber mit extra Samples
    i = 1
    for s1, s2, s3 in zip(shadow1.get_energy_list(), shadow2.get_energy_list(), shadow3.get_energy_list()):
        fileString += str(s1) +"\t" + str(s2) +"\t" + str(s3)
        fileString += "\n"
    file_name = "LSTM Mixture QAOA hidden 4 random graph1 layer1_1.txt"
    file_name = fileprefix + file_name
    file_name = cluster_string(file_name)
    f = open(file_name, "w")
    f.write(fileString)
    f.close()
    print("fertig!!")

#create_data_mit_abweichung_pool([0.5, 1, 1.5, 2, 2.5, 3, 3.5], [0.5, 1, 1.5, 2, 2.5, 3, 3.5], beta_grenzen=[1.5,2.5], gamma_grenzen=[1.5,2.5], showDia=True, savePath="LSTM Model2 parameter random pool_1", random_sets=10, max_iter=300, rndMitAnaly=True)
#writefile_QAOA(epoch=400, layers=1, shots=1024)
#writefile_QAOA_training_plus_NN(epoch=600, layers=4, shots=960, add_NN_samples=64, max_iter=80, fileprefix="896vs64")
#writefile_QAOA_training_plus_NN(epoch=600, layers=4, shots=896, add_NN_samples=128, max_iter=80, fileprefix="896vs128")
#writefile_QAOA_training_plus_NN(epoch=600, layers=4, shots=768, add_NN_samples=256, max_iter=80, fileprefix="768vs256")
#writefile_QAOA_training_plus_NN(epoch=600, layers=4, shots=512, add_NN_samples=512, max_iter=80, fileprefix="512vs512")
#writefile_QAOA_training_plus_NN(epoch=600, layers=4, shots=256, add_NN_samples=768, max_iter=80, fileprefix="256vs768")
#writefile_QAOA_training_plus_NN(epoch=600, layers=4, shots=128, add_NN_samples=896, max_iter=80, fileprefix="128vs896")
#writefile_QAOA_training_plus_NN(epoch=600, layers=4, shots=64, add_NN_samples=980, max_iter=80, fileprefix="64vs980")
#writefile_QAOA_training_plus_NN(epoch=600, layers=4, shots=32, add_NN_samples=992, max_iter=80, fileprefix="32vs992")
writefile_QAOA_training_plus_NN(epoch=600, layers=4, shots=16, add_NN_samples=1008, max_iter=80, fileprefix="16vs1008")
#writefile_QAOA_training_plus_NN(epoch=600, layers=4, shots=8, add_NN_samples=1016, max_iter=80, fileprefix="8vs1016")

#writefile(n = 100, epoch=200)

C:\Users\zazabla\AppData\Local\Temp\ipykernel_14760\1935846830.py:462: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:278.)
  x = torch.tensor(X, dtype=torch.float32)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\zazabla\anaconda3\envs\qiskit2\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Ple

Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s] tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[[1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1., 1.]],

        [[

NameError: name 'testsdf' is not defined